
# Digit Recognition


## Introduction

This notebook is concerned with explaining the Digit recognition script that I wrote in relation to the MNIST dataset.
In the previous notebook, I went through how to read in the MNIST dataset and also how to save the images to the comoputers file system. I have re used some of that code in this script, so this notebook will mainly be in relation to the parts of the script that have nothing to do with that.

## What is a neural network?

Artificial neural networks are essentially a computing system designed based of the  

In [1]:
import gzip
import numpy as np
import tensorflow as tf


In [2]:
import numpy as np
import keras as kr
import sklearn.preprocessing as pre
import sys
import gzip
import cv2
from PIL import Image

Using TensorFlow backend.
